# Proyecto 1

**Integrantes:**

- Daniel Cordoba Pulido

- Daniel Martínez Vega

# 1. Descripción
En este proyecto se crea un ambiente de desarrollo de machine learning
en el cual es posible la ingesta, validación y transformación de datos, demostrando capacidad de versionar los datos, código y ambiente de desarrollo.

En este proyecto se construyó un ambiente de desarrollo para realizar un pipeline de datos con estas etapas:

- Selección de características
- Ingesta del conjunto de datos
- Generació de las estadísticas del conjunto de datos
- Creación de un esquema según el conocimiento del dominio
- Creación de entornos de esquema
- Visualización de las anomalías del conjunto de datos
- Preprocesado, transformación e ingeniería de características
- Seguimiento de la procedencia del flujo de datos utilizando metadatos de ML

## Librerías importadas

In [3]:
import os # Para manejo de archivos y carpetas
import requests # Para importar el conjunto de datos
import pandas as pd #Para manejo de DataFrames
import sqlite3 # Para manejo de bases de datos

from sklearn.feature_selection import SelectKBest,f_classif # Para selección de características

In [56]:
# Para trabajar de forma interactiva con los componentes de TFX
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext


from tfx.components import CsvExampleGen # Generación de ejemplos
from tfx.components import StatisticsGen # Generación de Estadísticas
from tfx.components import ExampleValidator # Generación de Ejemplos
from tfx.components import SchemaGen # Generación de Esquemas de validación de datos
from tfx.components import Transform # Transformación de datos
from tensorflow_metadata.proto.v0 import schema_pb2 # Configuración de dominos de esquemas
from tfx.dsl.components.common.importer import Importer # Para importar componentes
from tfx.types import standard_artifacts

import tensorflow_data_validation as tfdv # Validación de datos

# 2. Descripción del dataset
Se utilizó una variante del conjunto de datos [Tipo de Cubierta Forestal](https://archive.ics.uci.edu/dataset/31/covertype) para entrenar un modelo que predice el tipo de cobertura forestal en función de variables cartográficas. Puede ver el código para preparar el conjunto de datos [aquí](https://github.com/GoogleCloudPlatform/mlops-on-gcp/blob/master/datasets/covertype/wrangle/prepare.ipynb)

## 2.1. Carga del Dataset

In [5]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [6]:
df=pd.read_csv("data/covertype/covertype_train.csv")

### Revisión rápida del DataFrame

Se hizo una pequeña revisión de los datos para comprobar fueron importados correctamente:

- El conjunto de datos sumistrado cuenta con 13 características (11 numéricas y 2 categóricas), donde la variable objetivo es **Cover_Type**.

- Gracias al procesamiento que se hizo del conjunto de datos, este no cuenta con valores nulos ni con registros duplicados

In [7]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

# 3. Selección de características
En esta estapa se realizó una selección de características sobre las variables numéricas usando **SelectKBest** de **Scikit-Learn**. Esta clase selecciona los features más relevantes respecto a la variable objetivo (**Cover_Type**).

Para este caso, se seleccionaron **los mejores 8 features** y la función estadística utilizada para calificar a los features fue **f_classif**, el cual realiza un análisis anova sobre los datos para puntuarlos.

In [9]:
# Separación de  las caracteristicas de entrada y de destino
X=df.drop("Cover_Type",axis=1)
y=df["Cover_Type"]

# Extracción de características numéricas
X_n=X.select_dtypes(include=['number'])

# Seleccción de las mejores caraterísticas con SelectKBest
selector=SelectKBest(f_classif, k=8)
X_new=selector.fit_transform(X_n, y)

## Características seleccionadas
Como se puede apreciar en la siguiente tabla, los features que fueron excluidos por **SelectKBest** fueron **Aspect** y **Hillshade_3pm**

In [10]:
pd.DataFrame({"Columnas":X_n.columns,"Retain":X_n.columns.isin(selector.get_feature_names_out())})

,Columnas,Retain
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


## Creación del nuevo DataFrame sin los parámetros excluidos por SelectKBest
Ahora, se crea una segunda versión del conjunto de datos en la que no se incluyen estas características excluidas.

In [11]:
new_dataframe=pd.DataFrame(X_new,columns=selector.get_feature_names_out())

In [12]:
new_dataframe.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,7,67,11,1015,233,234,1570
1,2876,18,485,71,2495,192,202,1557
2,3171,2,277,9,4374,213,237,1052
3,3087,13,190,31,4774,193,221,752
4,2835,10,212,41,3596,231,242,3280


In [13]:
new_dataframe["Wilderness_Area"]=df["Wilderness_Area"]
new_dataframe["Soil_Type"]=df["Soil_Type"]
new_dataframe["Cover_Type"]=y

In [14]:
new_dataframe.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,7,67,11,1015,233,234,1570,Commanche,C7202,1
1,2876,18,485,71,2495,192,202,1557,Commanche,C7757,1
2,3171,2,277,9,4374,213,237,1052,Rawah,C7745,0
3,3087,13,190,31,4774,193,221,752,Rawah,C7745,0
4,2835,10,212,41,3596,231,242,3280,Rawah,C4744,1


In [15]:
new_dataframe.to_csv('data/sel/data_selection.csv', index=False)

# 4. Pipeline de datos
En esta cuarta sección, se construyó un pipeline de datos encargado de la ingesta, validación y transformación de datos usando componentes de Tensorflow Extended (TFX)

## 4.1 Configuración del contexto interactivo
Primero que todo, para poder trabajar de forma interactiva dentro de un notebook con los componentes de TFX, se configuró un contexto interactivo. 

el parámetro **pipeline_root="meta"** define que los artefactos y sus metadatos serán almacenados en una carpeta llamada **meta**, donde se crea una base de datos sqlite con los metadatos de estos artefactos

In [16]:
metadata_path = 'meta/metadata.sqlite'

context = InteractiveContext(pipeline_root="meta")

## 4.2 Generación de ejemplos

Para la ingesta de datos, se implementó **CsvExampleGen** para convertir datos sin
procesar en TFRecords para un cálculo más rápido en las etapas posteriores del pipeline.

Este usa como entrada la ruta a la carpeta donde se encuentran los datos en formato CSV

**Nota:** para ejecutar este y los demas componentes de TFX utilizados en este notebook se utiliza **context.run()**

In [17]:
example_gen = CsvExampleGen(input_base='data/sel')
context.run(example_gen);

## 4.3 Generación de estadísticas
Una vez ralizada la ingesta de datos, se procede a calcular y visualizar las características y distribución de los datos, esto permite identificar visualmente el comportamiento de los features del conjunto de datos e identificar posibles anomalias que son resaltadas en color rojo.

In [18]:
stats_gen= StatisticsGen(examples=example_gen.outputs['examples'],)
context.run(stats_gen);

### Visualización de estadísticas

En esta ocasión, se rasalta la cantidad de valores en cero en la variable objetivo (Cover_Type) como un porblema potencial. Sin embargo, esta es una particularidad de esta variable que toma valores entre 0 y 6. 

**Nota:** Para poder visualizar este y otros componentes de TFX en el contexto interactivo se usa **context.show()**

In [19]:
context.show(stats_gen.outputs['statistics'])

## 4.4 Inferencia de Esquema
Una vez calculadas las estadísticas a partir del conjunto de datos, para poder automatizar la validación de datos para entrenar algun modelo o hacer inferencias, se genera un esquema usando **SchemaGen** a partir de las características.

In [20]:
schema_gen = SchemaGen(statistics=stats_gen.outputs['statistics'])
context.run(schema_gen);

### Visualización del esquema generado
Como se puede apreciar, para cada uno de los features se define el tipo de dato que debe tener, si es necesario y para los features categóricos los posibles valores que puede tomar.

In [21]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 4.5 Configuración del esquema
Ademas de configurar los posibles valores que pueden tomar las variables categóricas, en este apartado se definirá un domino con los valores que pueden tomar algunas de las variables numéricas del conjunto de datos

### Carga del esquema
Para poder modificar el esquema, primero lo almacenamos en una variable para poder trabajar con el.

In [22]:
schema_path = schema_gen.outputs['schema'].get()[0].uri + '/schema.pbtxt'
schema = tfdv.load_schema_text(schema_path)

### Modificación de dominios
Se definió los siguientes dominios para estos features usando **tfdv.set_domain()** y **schema_pb2.IntDomain()**
- Hillshade 9am: 0 to 255
- Hillshade Noon: 0 to 255
- Slope: 0 to 90
- Cover Type: 0 to 6

Además, la variable objetivo (Cover_Type) se declaró como una variable categórica con el parámetro **is_categorical=True** 

In [23]:
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(min=0, max=255))

In [24]:
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(min=0, max=255))

In [25]:
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(min=0, max=90))

In [26]:
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover Type', min=0, max=6, is_categorical=True))

### Actualización del esquema
Una vez ajustados los dominios de los features, se sobreescribe el esquema original para guardar los cambios.

In [27]:
tfdv.write_schema_text(schema, schema_path)

### Visualización del esquema actualizado
Se puede observar que en el esquema actualizado, para la columna Domain ya se encuentran los valores máximo y mínimo que pueden tomar als variables a las que se les definió un dominio

In [28]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 4.6 Entornos de esquema
El esquema que se ha ajustado funciona correctamente en los datos de entrenamiento, pero cuando se quiera hacer una validación de los datos con los que se quiera hacer inferencias, el esquema detectará la falta de la variable objetivo **(Cover_Type)** como una anomalía.

Para arreglar este problema, en esta sección se crean dos entornos para este esquema uno para datos de entrenamiento y otro para datos de inferencia. 

### Creación de subconjunto de inferencia
Para poder comprobar el correcto funcionamiento de los entornos de esquema, se crea un segundo dataframe que simula un conjunto de datos de inferencia el cual no cuenta con la variable objetivo

In [29]:
new_dataframe.drop("Cover_Type",axis=1).to_csv('data/inference/inference.csv', index=False)

### Validación del subconjunto de inferencia con esquema actual
Como se puede apreciar, el esquema actual detecta anomalías en el conjunto de inferencias ya que no cuentan con las etiquetas

In [31]:
serving_anomalies = tfdv.validate_statistics(statistics=tfdv.generate_statistics_from_csv("data/inference/inference.csv"), schema=schema)

tfdv.display_anomalies(serving_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


### Añadir entornos de esquema para los conjuntos de entrenamiento e inferencia
Para poder validar correctamente los datos de inferencia, se crearon dos entornos uno de TRAINING y otro de INFERENCIA para evaluar los datos entrantes en cada una de las ocasiones

In [32]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('INFERENCIA')

### Especificar que la salida ("Cover_Type") no está en el entorno de inferencia
Para que la falta de la variable objetivo no sea detactada como una anomalía por parte del entorno de INFERENCIA, se ajusta el feature para que no se tenga en cuenta en dicho entorno

In [33]:
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('INFERENCIA')

### Verificación de los entornos del esquema

#### Verificación de rangos de dominio

Primero, se veirfica que los dominios se han agregados correctamente en el esquema

In [34]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Verificación de la existencia de entornos
Lugo, se verifica que efectivamente se hayan creado los dos entornos para el esquema

In [35]:
# Entornos
schema.default_environment

['TRAINING', 'INFERENCIA']

#### Verificación de features no tenidos en cuenta en el entorno de inferencia
Además, se verifica que efectivamente en el entorno 'INFERENCIA', no se encuentre el feature 'Cover_Type'

In [36]:
for feature in schema.feature:
    if feature.not_in_environment:
        print(feature.name)

Cover_Type


#### Validación del conjunto de inferencia con el nuevo entorno
y por último, se verifica que no se detecte anomalías debido a la fallta del feature objetivo (Cover_Type) en el conjunto de datos de inferencia usando el entorno **INFERENCIA**

In [37]:
serving_anomalies = tfdv.validate_statistics(statistics=tfdv.generate_statistics_from_csv("data/inference/inference.csv"), schema=schema, environment='INFERENCIA')

tfdv.display_anomalies(serving_anomalies)

#### Guardar cambios
Para terminar, se sobreescribe el esquema para guardar los cambios.

In [38]:
tfdv.write_schema_text(schema, schema_path)

## 4.7 Generación de nuevas estadísticas usando el esquema actualizado
En esta sección se comprueba que los cambios realizados al esquema se han realizado correctamente visualizando las estadísticas generadas en el esquema usando el esquema actualizado.

El principal cambio que se puede observar al revisar las etadísticas con este nuevo esquema es que el feature objetivo (Cover_Type) ahora se encuentra en la sección de variables categóricas.

In [40]:
user_schema_importer = Importer(
    source_uri=schema_gen.outputs['schema'].get()[0].uri,
    artifact_type=standard_artifacts.Schema,
    reimport= True).with_id('schema_importer')

context.run(user_schema_importer)

compute_eval_stats = StatisticsGen(
      examples=example_gen.outputs['examples'],
      schema=user_schema_importer.outputs['result']
      )

context.run(compute_eval_stats);

In [41]:
context.show(compute_eval_stats.outputs['statistics'])

## 4.8 Comprobación de anomalías
También se puede realizar una comprobación de anomalias en el conjunto de datos usando **ExampleValidator()**. Dado que las anomalías son comprobadas en el mismo conjunto de datos con el que se creó el esquema, no fueron detectadas anomalías

In [42]:
validate_stats = ExampleValidator(
      statistics=compute_eval_stats.outputs['statistics'],
      schema=schema_gen.outputs['schema']
      )
context.run(validate_stats);

In [43]:
context.show(validate_stats.outputs['anomalies'])

## 4.9 Ingeniería de características
En esta sección se transforman los features de forma adecuada para entrenar e implementar un modelo, para esto, en la **sección 4.10**, se crea una función de preprocesamiento la cual a partir de los features de entrada especificados, les aplica una transformación y genera unos features de salida

## 4.10. Función de preprocesamiento

Para esta sección, en el archivo **prep.py** que se encuentra en el mismo directorio que este notebook se definió la función de preprocesamiento utilizada para hacer la transformación de datos. Esta función a partir de los datos de entrada, entrega un diccionario con los datos transformados para cada uno de los features

En la función de preprocesamiento creada se usan dos tipos de transformaciónes:

- Para las variables categóricas, se utiliza **tft.compute_and_apply_vocabulary()** para generar un vocabulario a partir de los datos de entrada y luego aplicar este vocabulario para transformar los valores de string en índices enteros. 

- Para las variables numéricas, se utiliza **tft.scale_to_0_1()** para escalar numéricamente los valores de una característica de entrada de manera que queden en el rango de 0 a 1.

- Por último, para la variable objetivo (Cover_Type), no se realizó ninguna transformación


El contenido de la función se presenta en el siguiente bloque de código:

In [44]:
def preprocessing_fn(inputs):
    
    outputs = {}
    
    # Transformación de features categóricos
    for categorical_feature in ['Wilderness_Area', 'Soil_Type']:
        outputs[categorical_feature] = tft.compute_and_apply_vocabulary(inputs[categorical_feature])
    
    # Transformación de features numéricos
    for numeric_feature in ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']:
        outputs[numeric_feature] = tft.scale_to_0_1(inputs[numeric_feature])
        
    outputs['Cover_Type'] = inputs['Cover_Type']
    
    return outputs

## 4.11 Transformación del dataset
En esta sección se usa el componente de TFX **Transform** para transformar el conjunto de datos usando la función de preprocesamiento definida en la sección anterior 

In [46]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn= "prep.preprocessing_fn"
)
context.run(transform);

### Visualización de estadísticas sobre el dataset con las transformaciones aplicadas
Por último, se visualiza las estadísticas de los nuevos datos transformados, lo más notorio es que ahora todos los features son numerícos y los features que originalmente eran numéricos, ahora us valores están escalados.

Para los features Soil_Type y Wilderness_Area se destaca como problema el porcentaje de valores en cero que toman estas variables, esto no es un problema dado que esto es debido a las transformaciones realizadas a estos features

In [47]:
transformed_statistics_gen = StatisticsGen(
    examples=transform.outputs['transformed_examples']
)

context.run(transformed_statistics_gen);

In [48]:
context.show(transformed_statistics_gen.outputs['statistics'])

# 5. Metadatos de aprendizaje automático

## Librerías importadas

In [49]:
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

## Conexíon con el almacen de metadatos
Al crear el contexto interactivo en la primera etapa de la sección 4 también se creó una base de datos sqlite en el mismo directorio, esta base de datos amacena los metadatos de cada uno de los artefactos creados en el contexto interactivo.

En esta seccion nos conectamos a este almacén de metadatos el cual se encuentra en la ruta **'meta/metadata.sqlite'** usando **MetadataStore()** en modo de lectura ya que no buscamos realizar cambios a este almacen.

In [50]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'meta/metadata.sqlite'
connection_config.sqlite.connection_mode = 1 

store = metadata_store.MetadataStore(connection_config)

## 5.1 Acceso a artefactos almacenados
Una vez realizada la conexión, en esta sección se hará una revisión de los componentes almacenadoe en esta base de datos

### Tipos de artefactos almacenados
Primero, se realizó una revisión de los distintos tipos de artefactos almacenados usando el método **.get_artifact_types()**. En la salida de este bloque se puede observar los artefactos que fueron generados durante el desarrollo de la sección 4

In [51]:
store.get_artifact_types()

[id: 14
 name: "Examples"
 properties {
   key: "span"
   value: INT
 }
 properties {
   key: "split_names"
   value: STRING
 }
 properties {
   key: "version"
   value: INT
 }
 base_type: DATASET,
 id: 16
 name: "ExampleStatistics"
 properties {
   key: "span"
   value: INT
 }
 properties {
   key: "split_names"
   value: STRING
 }
 base_type: STATISTICS,
 id: 18
 name: "Schema",
 id: 21
 name: "ExampleAnomalies"
 properties {
   key: "span"
   value: INT
 }
 properties {
   key: "split_names"
   value: STRING
 },
 id: 23
 name: "TransformGraph",
 id: 24
 name: "TransformCache"]

### Artefactos de tipo Esquema
Por otra parte, también se puede observar para un determinado tipo de artefacto, todos los artefactos que fueron construidos junto a sus propiedades usando el método **.get_artifacts_by_type()**

In [52]:
store.get_artifacts_by_type("Schema")

[id: 3
 type_id: 18
 uri: "meta/SchemaGen/schema/3"
 custom_properties {
   key: "name"
   value {
     string_value: "schema:2024-03-04T03:27:19.947256"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "SchemaGen"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.12.0"
   }
 }
 state: LIVE
 name: "schema:2024-03-04T03:27:19.947256"
 create_time_since_epoch: 1709522840704
 last_update_time_since_epoch: 1709522840794,
 id: 4
 type_id: 18
 uri: "meta/SchemaGen/schema/3"
 custom_properties {
   key: "is_external"
   value {
     int_value: 1
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.12.0"
   }
 }
 create_time_since_epoch: 1709522864225
 last_update_time_since_epoch: 1709522864425,
 id: 10
 type_id: 18
 uri: "meta/Transform/pre_transform_schema/7"
 custom_properties {
   key: "name"
   value

## 5.2 y 5.3 Seguimiento de artefactos y Obtención de artefactos principales
En esta última sección se diseñó una función con la cual se puede acceder a los artefactos a partir de los cuales un determinado componente fue generado

### Función de obtención de artefactos

In [53]:
def obtener_artefactos_principales(store, output_artifact_id):
    """
    Devuelve los artefactos usados para generar el artefacto de salida especificado.
    
    Args:
    store: Un store Metadata conectada a ML Metadata.
    output_artifact_id: El ID del artefacto de salida.
    
    Returns:
    lista de artefactos de entrada usados para generar el artefacto de salida especificado.
    """
    input_artifacts = []

    # Encuentra todos los eventos asociados con el artefacto de salida
    events = store.get_events_by_artifact_ids([output_artifact_id])

    for event in events:
        if event.type == metadata_store_pb2.Event.OUTPUT:  # Busca eventos donde el artefacto es de salida
            # Encuentra la ejecución asociada con este evento
            execution_id = event.execution_id
            # Encuentra todos los eventos asociados con la ejecución para identificar artefactos de entrada
            related_events = store.get_events_by_execution_ids([execution_id])

            for rel_event in related_events:
                if rel_event.type == metadata_store_pb2.Event.INPUT:  # Busca eventos donde los artefactos son de entrada
                    artifact_id = rel_event.artifact_id
                    # Recupera el artefacto de entrada y lo añade a la lista
                    artifact = store.get_artifacts_by_id([artifact_id])[0]
                    input_artifacts.append(artifact)

    return input_artifacts

### Testeo de la función

#### Examples

In [54]:
obtener_artefactos_principales(store, 2)[0].uri

'meta/CsvExampleGen/examples/1'

#### Schema

In [55]:
obtener_artefactos_principales(store, 5)[1].uri

'meta/SchemaGen/schema/3'